In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

24/11/04 21:38:45 WARN Utils: Your hostname, lucaires-IdeaPad-3-15ALC6 resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp1s0)
24/11/04 21:38:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/04 21:38:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/11/04 21:38:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [3]:
df_tweet = spark.read.json("/home/lucaires/Documentos/airflow_twitter/datalake/silver/twitter_datascience/tweet")

In [4]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [5]:
from pyspark.sql import functions as f

In [6]:
tweet_conversas = df_tweet.alias("tweet")\
                    .groupBy(f.to_date("created_at").alias("created_date"))\
                        .agg(
                            f.countDistinct("author_id").alias("n_tweets"),
                            f.sum("like_count").alias("n_like"),
                            f.sum("quote_count").alias("n_quote"),
                            f.sum("reply_count").alias("n_reply"),
                            f.sum("retweet_count").alias("n_retweet")         
                        ).withColumn("weekday", f.date_format("created_date", "E"))

In [7]:
tweet_conversas.show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2024-11-02|      29|  4101|   3690|   3975|     4341|    Sat|
|  2024-11-04|      10|   435|    398|    580|      519|    Mon|
|  2024-11-03|      24|  4149|   3405|   3573|     3936|    Sun|
+------------+--------+------+-------+-------+---------+-------+

